In [2]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import pandas as pd
import os
import sys
import datetime

root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)
    

load_dotenv()

True

In [2]:
import tensorflow as tf
tf.__version__, tf.config.list_physical_devices('GPU')

('2.10.1', [])

In [3]:
from training.utils import date_range, df_to_X_y

df_train = pd.concat([
    pd.read_csv(f'../datasets/d{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2019, 11, 1), 
        datetime.date(2020, 1, 15),
        datetime.timedelta(days=1),
    )
])
df_val = pd.concat([
    pd.read_csv(f'../datasets/d{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2020, 1, 15), 
        datetime.date(2020, 2, 15),
        datetime.timedelta(days=1),
    )
])

X_train, y_train = df_to_X_y(df_train)
X_val, y_val = df_to_X_y(df_val)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((1494497, 124), (1494497, 1), (683009, 124), (683009, 1))

In [5]:
from keras.models import Sequential, clone_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam


model = Sequential()
model.add(InputLayer((124, 1)))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(8, 'relu'))
model.add(Dense(1, 'linear'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 122, 64)           256       
                                                                 
 flatten (Flatten)           (None, 7808)              0         
                                                                 
 dense (Dense)               (None, 8)                 62472     
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 62,737
Trainable params: 62,737
Non-trainable params: 0
_________________________________________________________________


In [6]:
model_view = clone_model(model)
cp_view = ModelCheckpoint('../model/d-view.keras', save_best_only=True)
model_view.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_view.fit(X_train, y_train[:, 0], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_view])

model_cart = clone_model(model)
cp_cart = ModelCheckpoint('../model/d-cart.keras', save_best_only=True)
model_cart.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_cart.fit(X_train, y_train[:, 1], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_cart])

model_remove_from_cart = clone_model(model)
cp_remove_from_cart = ModelCheckpoint('../model/d-remove_from_cart.keras', save_best_only=True)
model_remove_from_cart.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_remove_from_cart.fit(X_train, y_train[:, 2], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_remove_from_cart])

model_purchase = clone_model(model)
cp_purchase = ModelCheckpoint('../model/d-purchase.keras', save_best_only=True)
model_purchase.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_purchase.fit(X_train, y_train[:, 3], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_purchase])

Epoch 1/10
46704/46704 [==============================] - 169s 4ms/step - loss: 17.0464 - root_mean_squared_error: 4.1287 - val_loss: 80.4579 - val_root_mean_squared_error: 8.9698
Epoch 2/10
46704/46704 [==============================] - 159s 3ms/step - loss: 14.9516 - root_mean_squared_error: 3.8667 - val_loss: 83.7335 - val_root_mean_squared_error: 9.1506
Epoch 3/10
46704/46704 [==============================] - 166s 4ms/step - loss: 14.4258 - root_mean_squared_error: 3.7981 - val_loss: 87.1477 - val_root_mean_squared_error: 9.3353
Epoch 4/10
46704/46704 [==============================] - 167s 4ms/step - loss: 14.2980 - root_mean_squared_error: 3.7813 - val_loss: 78.3587 - val_root_mean_squared_error: 8.8520
Epoch 5/10
46704/46704 [==============================] - 178s 4ms/step - loss: 13.9401 - root_mean_squared_error: 3.7336 - val_loss: 79.6208 - val_root_mean_squared_error: 8.9230
Epoch 6/10
46704/46704 [==============================] - 201s 4ms/step - loss: 13.7476 - root_mean_

In [11]:
model_view.save('../model/saved/d-view.keras')
model_cart.save('../model/saved/d-cart.keras')
model_remove_from_cart.save('../model/saved/d-remove_from_cart.keras')
model_purchase.save('../model/saved/d-purchase.keras')